In [34]:
import pandas as pd
import math
import copy
df = pd.read_csv('data.csv', sep=',')
df

,Alternatiive,Bar,Weekend,Hungry,Patrons,Price,Rain,Reservation,Type,Estimate,Goal
0,True,False,False,True,Some,$$$,False,True,French,0->10,True
1,True,False,False,True,Full,$,False,False,Thai,30->60,False
2,False,True,False,False,Some,$,False,False,Burger,0->10,True
3,True,False,True,False,Full,$,False,False,Thai,10->30,True
4,True,False,True,False,Full,$$$,False,True,French,>60,False
5,False,True,False,True,Some,$$,True,True,Italian,0->10,True
6,False,True,False,False,None,$,True,False,Burger,0->10,False
7,False,False,False,False,Some,$$,True,True,Thai,0->10,True
8,False,True,True,False,Full,$,True,False,Burger,>60,False
9,True,True,False,True,Full,$$$,False,True,Italian,10->30,False


Class của Descition bao gồm tất cả thuộc tính cơ bản mà dữ liệu đang có

In [35]:
class Decision:
    def ListCreate(self):
        self.Features = [self.Alternative, self.Bar, self.Weekend,
                        self.Hungry, self.Patrons, self.Price,
                        self.Rain, self.Reservation, self.Type,
                        self.Estimate, self.Goal]

    def __init__(self, li):
        self.Alternative, self.Bar, self.Weekend = [li[i] for i in range(3)]
        self.Hungry, self.Patrons, self.Price = [li[i] for i in range(3,6)]
        self.Rain, self.Reservation, self.Type = [li[i] for i in range(6,9)]
        self.Estimate, self.Goal, self.listValues = li[9], li[10], li
        self.ListCreate()

    def setFeatureAt(self, index, value):
        self.Features[index] = value

    def getFeatureAt(self, index):
        return self.Features[index]
        
    def __str__(self):
        str = f"Alternative: {self.Alternative}\n" + f"Bar: {self.Bar}\n" + f"Weekend: {self.Weekend}\n"
        str += f"Hungry: {self.Hungry}\n" + f"Patrons: {self.Patrons}\n" + f"Price: {self.Price}\n"
        str += f"Rain: {self.Rain}\n" + f"Reservation: {self.Reservation}\n" + f"Type: {self.Type}\n"
        str += f"Estimate: {self.Estimate}\n" + f"Reservation: {self.Goal}\n"
        return str

    def getNameAt(index):
        Names = df.columns
        return Names[index]

    def getIndexByName(string):
        Names = df.columns
        return int([index for index in range(len(Names)) if Names[index]==string][0])
    
    def getLenHeader():
        return len(df.columns)


Hàm đọc file vào mảng objects

In [36]:
Decides = list()
alist = [line.rstrip() for line in open('data.csv')]
alist.pop(0)

for line in alist:
    values = line.split(',')
    decide = Decision(values)
    Decides.append(decide)

Hàm Entropy tính mức độ hỗn tạp của 1 mảng giá trị

In [37]:
def Entropy(listValue):
    total = 0
    for value in listValue:
        try: total+=value*math.log2(value)
        except: return 0
    return -total

Hàm AveEntropies tính trung bình các hỗn số

In [38]:
#input must be list of pairt, which contain Entropy and percentage of that value
def AveEntropies(PairValue):
    return sum([value[0]*value[1] for value in PairValue])

In [39]:
def InformationGain(EntropyGoal, AE):
    return EntropyGoal - AE

Hàm hoàn đổi các chỉ số hiện tại của mảng về tỷ lệ xác suất

In [40]:
def ConvertToProbability(ListObject):
    listProb = dict()
    for value in ListObject:
        if value in listProb:
            listProb[value] += 1
        else:
            listProb[value] = 1
    listValues = list(listProb.values())
    S = sum([value for value in listValues])
    return [value/S for value in listValues]

Hàm sơ lượt kiểm tra và cắt các giá trị records dựa theo điều kiện bất kỳ (nếu có)

In [41]:
def splitRecord(listDecisions, Condition=dict()):
    def checkCondition(decide):
        for key in Condition:
            if (Condition[key]!=decide.getFeatureAt(key)):
                return False
        return True

    result = list()
    for decide in listDecisions:
        if (checkCondition(decide)==False): continue
        result.append(list())
        for i in range(len(decide.Features)):
            result[len(result)-1].append(decide.getFeatureAt(i))
    return result

Hàm tìm ra Entropy ít nhất trong các chuỗi dữ liệu cho vào

In [42]:
def FindBestEntropy(Records):
    bestIndex, minEntropy = -1, 99999
    for j in range(len(Records[0])): 
        x = [Records[i][j] for i in range(len(Records))]
        x = ConvertToProbability(x)
        print(f"{Decision.getNameAt(j)}: {x}")
        print(f"=> {Entropy(x)}")
        minEntropy, bestIndex = (Entropy(x), j) if (Entropy(x) < minEntropy) else (minEntropy, bestIndex)
    return bestIndex, minEntropy

In [43]:
def getDistinctValue(feature):
    result = list()
    for value in feature:
        if value not in result:
            result.append(value)
    return result

In [44]:
goal = Decision.getLenHeader()-1
goal_list = [decide.getFeatureAt(goal) for decide in Decides]
goal_cases = getDistinctValue(goal_list)
goal_entropy = Entropy(ConvertToProbability(goal_list))

HighestIG, bestFeature = 0, "undefined"
# lặp for cho từng feature trong dữ liệu
for feature_index in range(Decision.getLenHeader()-1):
    feature_list = [decide.getFeatureAt(feature_index) for decide in Decides]   
    feature_cases = getDistinctValue(feature_list)

    probability_tuple = list()
    entropy_list = list()
    numberApperence = list()

    # lập for cho từng kiểu kết quả trong feature đó
    for fcase in feature_cases: 
        total_fcase = len([decide.getFeatureAt(feature_index) 
                        for decide in Decides
                        if decide.getFeatureAt(feature_index)==fcase])
        
        # lặp for cho từng kiểu kết quả trong goal
        for gcase in goal_cases:
            Condition = {goal:gcase, feature_index:fcase}
            Records = splitRecord(Decides, Condition)
            probability_tuple.append((len(Records), total_fcase))

    # tách các tỉ lệ tính được thành từng nhòm và tính entropy        
    totalGCase = len(goal_cases)
    for i in range(0, len(probability_tuple), totalGCase):
        x = [probability_tuple[j][0] for j in range(i, i+totalGCase)]
        numberApperence.append(int([probability_tuple[j][1] for j in range(i, i+1)][0]))
        x = [value/sum(x) for value in x]
        entropy_list.append(Entropy(x))

    pairs = [(x, y/sum(numberApperence)) for x, y in zip(entropy_list, numberApperence)]
    IG = InformationGain(goal_entropy ,AveEntropies(pairs))
    HighestIG,bestFeature  = (IG,Decision.getNameAt(feature_index)) if IG>=HighestIG else (HighestIG,bestFeature)
print(f"{bestFeature} is the best feature of this data")

Patrons is the best feature of this data
